In [1]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")

base_correos = 'bases/directorios/Base Correos.xlsx'
directorios_oficinas = 'bases/directorios/T&C - Lista de Gof y Sgof.csv'

In [2]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
df_base = pd.read_excel(base_correos, dtype={'OFICINA': str, 'ID_OFICINA': str})
df_base.columns = clean_columns(df_base.columns)

df_base.rename(columns={
    'OFICINA': 'ID_OFICINA', 
    'CORREOS': 'CORREO',
    'CARGO': 'PUESTO',
    }, inplace=True)

df_base['CORREO'] = df_base['CORREO'].str.upper()

df_base.dropna(inplace=True)

print(df_base.shape)
df_base.head(3)

In [ ]:
df_directorio = pd.read_csv(directorios_oficinas, sep=';', dtype={'ID Oficina': str}, encoding='utf-8')
df_directorio.columns = clean_columns(df_directorio.columns)

df_directorio.rename(columns={
    'OFICINA': 'NOMBRE_OFICINA',
    }, inplace=True)

cols_req = ['ID_OFICINA', 'NOMBRE_OFICINA', 'REGISTRO', 'NOMBRE', 'CORREO', 'PUESTO']
df_directorio = df_directorio[cols_req]

df_directorio.dropna(inplace=True)

df_directorio = df_directorio[df_directorio['PUESTO'].str.contains('GERENTE', case=False)].reset_index(drop=True)

df_directorio['PUESTO'] = df_directorio['PUESTO'].replace({
    '(?i).*SUB GERENTE.*': 2,
    '(?i).*GERENTE.*': 1
    }, regex=True)

df_directorio.sort_values(by=['ID_OFICINA','PUESTO'], inplace=True, ignore_index=True)
df_directorio.reset_index(drop=True, inplace=True)

print(df_directorio.shape)
df_directorio.head(3)

In [ ]:
def concatenar_correos(grupo: pd.DataFrame) -> pd.Series:
    gerente_correo = grupo.loc[grupo['PUESTO'] == 1, 'CORREO'].values
    subgerente_correo = grupo.loc[grupo['PUESTO'] == 2, 'CORREO'].values
    
    if len(gerente_correo) > 0 and len(subgerente_correo) > 0:
        correos_concatenados = gerente_correo[0] + ', ' + subgerente_correo[0]
    elif len(gerente_correo) > 0:
        correos_concatenados = gerente_correo[0]
    else:
        correos_concatenados = subgerente_correo[0]
    
    return pd.Series({
        'ID_OFICINA': grupo['ID_OFICINA'].iloc[0],
        'PUESTO': 'GERENTE DE OFICINA',
        'CORREO': correos_concatenados,
        'REGISTRO': grupo['REGISTRO'].iloc[0],
        'NOMBRE': grupo['NOMBRE'].iloc[0]
        })

df_directorio_final = df_directorio.groupby('ID_OFICINA').apply(concatenar_correos).reset_index(drop=True)

print(df_directorio_final.shape)
df_directorio_final.head(3)

In [ ]:
df_actualizado = pd.merge(df_base, df_directorio_final, on='ID_OFICINA', how='left', suffixes=('_x', ''))

df_actualizado['REGISTRO'] = df_actualizado['REGISTRO'].fillna(df_actualizado['REGISTRO_x'])
df_actualizado['NOMBRE'] = df_actualizado['NOMBRE'].fillna(df_actualizado['NOMBRE_x'])
df_actualizado['PUESTO'] = df_actualizado['PUESTO'].fillna(df_actualizado['PUESTO_x'])
df_actualizado['CORREO'] = df_actualizado['CORREO'].fillna(df_actualizado['CORREO_x']).str.upper()
df_actualizado['ID_OFICINA'] = df_actualizado['ID_OFICINA'].astype(str).str.zfill(4)

df_actualizado.drop(columns=['REGISTRO_x', 'NOMBRE_x', 'PUESTO_x', 'CORREO_x'], inplace=True)

correos_nuevos = ['MIRKO.IRIARTE@BBVA.COM', 'PAULA.ZUNIGA@BBVA.COM']

def agregar_correos(lista_correos: list) -> str:
    for correo in correos_nuevos:
        if correo not in lista_correos:
            lista_correos.append(correo)
    return ', '.join(lista_correos)

df_actualizado['CORREO'] = df_actualizado['CORREO'].str.split(', ').apply(agregar_correos)

cols_req_final = ['ID_OFICINA', 'CORREO', 'TERRITORIO', 'NOMBRE_OFICINA', 'PUESTO', 'REGISTRO', 'NOMBRE']
df_actualizado = df_actualizado[cols_req_final]
print(df_actualizado.shape)
df_actualizado.head(3)

In [7]:
df_actualizado.to_excel(base_correos, index=False)
os.startfile(os.path.abspath(base_correos))